<a href="https://colab.research.google.com/github/Taeho-Kim-0322/Deep_Learning_Start/blob/master/%EC%8B%A4%EC%8A%B5_7_TransferLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#기존의 만들어진 모델을 가져와서 활용하자


# Inference 환경구성(TF Hub와 Transfer Learning)

## TF Hub 설치하기


In [ ]:
import tensorflow as tf 
import numpy as np
import matplotlib.pylab as plt
from tensorflow.keras.layers import Dense

In [ ]:
!pip install -U tf-hub-nightly
import tensorflow_hub as hub


## ImageNet Classifier


### Classifier 다운로드

Transfer Learning을 위한 네트워크 로드를 위해 `hub.module`을, 그리고 하나의 keras 층으로 감싸기 위해 `tf.keras.layers.Lambda`를 사용한다.

TF 2.0에서 사용할 수 있는 Image Classifier는 [이 링크](https://tfhub.dev/s?q=tf2&module-type=image-classification)에서 확인할 수 있다.

In [ ]:
IMAGE_SHAPE = (224, 224)

mobilenet_url ="https://tfhub.dev/google/tf2-preview/mobilenet_v2/classification/2"
# 
mobilenet = tf.keras.Sequential([
    hub.KerasLayer(mobilenet_url, input_shape=IMAGE_SHAPE+(3,))
])

## Simple Transfer Learning


TF Hub를 사용하면 사용자 개인의 데이터셋의 클래스를 인식하기 위한 top layer fine tuning을 쉽게 수행할 수 있다.

### Dataset

이 예제를 살펴보기 위해, TF의 flowers 데이터셋을 사용할 것이다.

In [ ]:
data_root = tf.keras.utils.get_file(
  'flower_photos','https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz',
   untar=True)

`tf.keras.preprocessing.image.image.ImageDataGenerator`를 사용하여 데이터를 로드

> label : ['sunflowers', 'daisy', 'roses', 'tulips', 'dandelion']

In [ ]:
image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255)
image_data = image_generator.flow_from_directory(str(data_root), batch_size=32, target_size=IMAGE_SHAPE)

plt.imshow(image_data[0][0][0]) 

### Image Batch에 대한 Classifier 실행

이제 이미지 배치에 대한 분류기를 실행하고, 얼마나 많은 예측들이 이미지에 잘 들어맞는지 확인해보자.

In [ ]:
for image_batch, label_batch in image_data:
  print("Image batch shape: ", image_batch.shape)
  print("Label batch shape: ", label_batch.shape)
  break

In [ ]:
# 추론시키기
result_batch = mobilenet.predict(image_batch) 

# 이미지넷 라벨명(1000개 카테고리) 다운로드
labels_path = tf.keras.utils.get_file('ImageNetLabels.txt','https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt')
imagenet_labels = np.array(open(labels_path).read().splitlines())

# 추론 결과에 이미지넷 라벨명을 매칭
predicted_class_names = imagenet_labels[np.argmax(result_batch, axis=-1)] 

In [ ]:
#결과 시각화
plt.figure(figsize=(10,9))
plt.subplots_adjust(hspace=0.5)
for n in range(30):
  plt.subplot(6,5,n+1)
  plt.imshow(image_batch[n])
  plt.title(predicted_class_names[n])
  plt.axis('off')
_ = plt.suptitle("ImageNet predictions")

결과가 완벽하진 않지만, 모델이 ("daisy"를 제외한) 모든 것을 대비해서 학습된 클래스가 아니라는 것을 고려하면 꽤나 합리적이다.

하지만 결코 만족스럽진 않다...

### Download Headless Model

Image feature vector를 추출하기 위한 TF Hub 모델은 [이 링크](https://tfhub.dev/s?module-type=image-feature-vector&q=tf2)에서 확인할 수 있다.

Feature extractor를 만들어보자.

### Classifier head를 붙이기

이제 mobilenet feature extractor 뒤에 새로운 classifier layer를 추가하자.

In [ ]:
mobilenet_feature_url = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/2" 

new_model = tf.keras.Sequential([
  hub.KerasLayer(mobilenet_feature_url, input_shape=(224,224,3)),
  Dense(image_data.num_classes, activation='softmax')
])

new_model.summary() # 다운로드받은 mobilenet feature extractor는 non-trainable임을 알 수 있다



### 모델 학습

학습과정의 환경을 설정하기 위해 compile한다.

In [ ]:
new_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

2 epoch만 학습시켜보자.

학습과정을 시각화하기 위해 아래 방법을 사용하면 Epoch단위가 아니라 배치 단위의 loss와 accuracy를 기록할 수 있다.

In [ ]:
class CollectBatchStats(tf.keras.callbacks.Callback):
  def __init__(self):
    self.batch_losses = []
    self.batch_acc = []

  def on_train_batch_end(self, batch, logs=None):
    self.batch_losses.append(logs['loss'])
    self.batch_acc.append(logs['acc'])
    self.model.reset_metrics()

In [ ]:
steps_per_epoch = np.ceil(image_data.samples/image_data.batch_size)

batch_stats_callback = CollectBatchStats()

history = new_model.fit(image_data, epochs=2, steps_per_epoch=steps_per_epoch, callbacks = [batch_stats_callback], verbose=1)

In [ ]:
plt.figure()
plt.ylabel("Loss")
plt.xlabel("Training Steps")
plt.ylim([0,2])
plt.plot(batch_stats_callback.batch_losses)

In [ ]:
plt.figure()
plt.ylabel("Accuracy")
plt.xlabel("Training Steps")
plt.ylim([0,1])
plt.plot(batch_stats_callback.batch_acc)

### Prediction 확인

모델을 통해 이미지 배치를 실행시킨 뒤 튀어나온 인덱스들을 클래스 이름으로 바꾼다.

In [ ]:
# 라벨 불러오기
class_names = np.array(['Daisy', 'Dandelion', 'Roses', 'Sunflowers', 'Tulips'])

# 새로운 모델로 추론
predicted_batch = new_model.predict(image_batch)
predicted_id = np.argmax(predicted_batch, axis=-1)

# 추론 결과를 라벨 정보와 매핑
predicted_label_batch = class_names[predicted_id]

# 실제 라벨 (Ground Truth)
true_label_id = np.argmax(label_batch, axis=-1)

결과를 시각화한다

In [ ]:
plt.figure(figsize=(10,9))
plt.subplots_adjust(hspace=0.5)
for n in range(30):
  plt.subplot(6,5,n+1)
  plt.imshow(image_batch[n])
  color = "green" if predicted_id[n] == true_label_id[n] else "red"
  plt.title(predicted_label_batch[n].title(), color=color)
  plt.axis('off')
_ = plt.suptitle("Model predictions (green: correct, red: incorrect)")

## Model export하기

학습시킨 모델을 저장해보자.

In [ ]:
model_name = '2020XXXX_model'

export_path = "/tmp/saved_models/"+model_name

new_model.save(export_path)
 

Export된 모델을 다시 로딩할 수 있고, 이는 동일한 결과를 도출한다.

In [ ]:
reloaded_model = tf.keras.models.load_model(export_path)

reloaded_result = reloaded_model.predict(image_batch)

In [ ]:

# 학습된 모델 추론 결과
predicted_batch = new_model.predict(image_batch)
predicted_id = np.argmax(predicted_batch, axis=-1)
predicted_label_batch = class_names[predicted_id]
print(predicted_label_batch)

# 학습된 모델을 저장했다가 불러와 추론한 결과

reloaded_predicted_batch = reloaded_model.predict(image_batch)
reloaded_predicted_id = np.argmax(predicted_batch, axis=-1)
reloaded_predicted_label_batch = class_names[predicted_id]
print(reloaded_predicted_label_batch)

저장된 모델로부터 추론을 수행할 수도 있고,[TFLite](https://www.tensorflow.org/lite/convert/) 나 [TFjs](https://github.com/tensorflow/tfjs-converter) 로 변환할 수 있다.
